# MACHINE LEARNING NÃO SUPERVISIONADO - CLUSTERING

Este projeto tem por objetivo desenvolver um algoritmo de Machine Learning para agrupar clientes do shopping.

os dados foram extraidos do Kaggle: 
https://www.kaggle.com/shwetabh123/mall-customers

In [1]:
# Importando as bibliotecas
import pandas as pd
import numpy as np

In [2]:
# Importando a biblioteca "warnings" para ignorar mensagens de erro
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importando o arquivo CSV e criando um dataframe
df = pd.read_csv("/content/drive/MyDrive/MACHINE_LEARNNING/Mall_Customers.csv", sep=",", encoding="iso-8859-1")

In [4]:
# Visualizando as 5 primeiras linhas do dataframe
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


# ATRIBUTOS:
* CustomerID: Identificação do cliente
* Genre: Gênero
* Age: Idade
* Annual Income (k$): Rendimento anual
* Spending Score (1-100): Pontuação de gastos

# Visualização gráfica dos dados

## Idade

In [5]:
# Importando o Plotly para criar representações gráficas dos atributos
import plotly.express as px

In [6]:
# Criando um histograma com o atributo Age ( Idade )
hist = px.histogram( df, x = "Age", nbins = 60)
hist.update_layout (width = 600, height = 400, title_text = "Distribuição de Idades")
hist.show()

O gráfico mostra uma distribuição diversificada entre as idades, sendo a 
menor idade 18 anos com 4 contagens e a idade máxima 70 anos com 2 contagens.

## Gênero

In [7]:
# Agora veremos as distribuições de gênero
hist = px.histogram( df, x = "Genre", nbins = 60)
hist.update_layout (width = 600, height = 400, title_text = "Distribuição de gênero")
hist.show()
df["Genre"].value_counts() # contagem por tipo

Female    112
Male       88
Name: Genre, dtype: int64

In [8]:
# Visualizando a quantidade de linhas e colunas
df.shape

(200, 5)

# Exploração e tratamento dos dados

## Alterando o nome das colunas

In [9]:
df.head()

,CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [10]:
# Alterando o nome da coluna Genre para Gênero
df.rename(columns={"Genre":"genero"},inplace=True)

In [11]:
# Alterando o nome da coluna Age para Idade
df.rename(columns={"Age":"idade"},inplace=True)

In [12]:
# Alterando o nome da coluna Annual Income (k$) para Rendimento 
df.rename(columns={"Annual Income (k$)":"rendimento"},inplace=True)

In [13]:
# Alterando o nome da coluna Spending Score (1-100) para Pontuação
df.rename(columns={"Spending Score (1-100)":"pontuacao"},inplace=True)

In [14]:
df.head()

,CustomerID,genero,idade,rendimento,pontuacao
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [15]:
# Obs.: não alterei a coluna CustomerID porque ela será irrelevante e será excluída

## Valores Missing ( NAN )

In [16]:
df.isnull().sum()

CustomerID    0
genero        0
idade         0
rendimento    0
pontuacao     0
dtype: int64

Verificamos que não existem valores nulos.

## Análise dos tipos de atributos:
* object = string
* int64 = números inteiros
* float64 = números reais
* complex = números complexos


Obs.: Lembrando que não é possivel desenvolver modelos de machine learning usando atributos do tipo OBJECT

In [17]:
df.dtypes

CustomerID     int64
genero        object
idade          int64
rendimento     int64
pontuacao      int64
dtype: object

## Dados estatísticos

In [18]:
df.describe()

,CustomerID,idade,rendimento,pontuacao
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


Podemos verificar que:
* A amostra possui 200 elementos
* A média de idade dos frequentadores do shopping fica em 38, e a mediana 36
* A idade mínima é 18 e a máxima é 70
* O rendimento máximo encontrado foi de $137000
* A pontuação média fica em 50.2

## Análise de Outliers ( Dados discrepantes )

Para essa análise utilizamos o gráfico boxplot

In [19]:
# Importando o Plotly para criar representações gráficas dos atributos
import plotly.express as px

In [20]:
boxplot = px.box( df, y = "rendimento")
boxplot.show()

Obs.: Rendimentos possui um valor fora do padrão de 137 mil, acima do valor máximo de 129 mil. Por não ser um valor tão distante podemos manter no modelo.

In [21]:
boxplot = px.box( df, y = "idade")
boxplot.show()

In [22]:
boxplot = px.box( df, y = "pontuacao")
boxplot.show()

Nenhum dado discrepante encontrado nos outros atributos.

# Pré-processamento

O pré-processamento é uma etapa fundamental que pode melhorar a performance dos algoritmos de análise, através da redução de dimensionalidade e eliminação de ruidos que interfiram no funcionamento dos algoritmos. 

In [23]:
df.head()

,CustomerID,genero,idade,rendimento,pontuacao
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


## Excluindo o atributo CustomerID

O atributo CustomerID não possui relevância para o modelo pois ele apenas emumera os elementos. Deste modo ele deve ser excluído.

In [24]:
# Criando um novo dataframe para receber os dados, excluindo o atributo CustomerID.

In [25]:
df2 = df.drop(labels="CustomerID",axis=1) # lembrando que axis significa eixo e 1 equivale a coluna.
df2.head()

,genero,idade,rendimento,pontuacao
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


## Transformando dados categóricos em dados numéricos

In [26]:
df2["genero"].replace({"Female":0,"Male":1},inplace=True) # Inplace = True altera os dados de forma permanente no dataframe atual
df2.head()

,genero,idade,rendimento,pontuacao
0,1,19,15,39
1,1,21,15,81
2,0,20,16,6
3,0,23,16,77
4,0,31,17,40


## Escalonamento

O objetivo do escalonamento é mudar os valores das colunas numéricas no conjunto de dados para usar uma escala comum, sem distorcer as diferenças nos intervalos de valores nem perder informações. O escalonamento ( normalização ) também é necessário para alguns algoritmos para modelar os dados corretamente.
O escalonamento considera média próxima de zero e desvio padrão próximo de 1.

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
escala = StandardScaler()
df2_esc = escala.fit_transform(df2) # Criando um novo dataframe com os dados escalonados

In [29]:
df2_esc

array([[ 1.12815215, -1.42456879, -1.73899919, -0.43480148],
       [ 1.12815215, -1.28103541, -1.73899919,  1.19570407],
       [-0.88640526, -1.3528021 , -1.70082976, -1.71591298],
       [-0.88640526, -1.13750203, -1.70082976,  1.04041783],
       [-0.88640526, -0.56336851, -1.66266033, -0.39597992],
       [-0.88640526, -1.20926872, -1.66266033,  1.00159627],
       [-0.88640526, -0.27630176, -1.62449091, -1.71591298],
       [-0.88640526, -1.13750203, -1.62449091,  1.70038436],
       [ 1.12815215,  1.80493225, -1.58632148, -1.83237767],
       [-0.88640526, -0.6351352 , -1.58632148,  0.84631002],
       [ 1.12815215,  2.02023231, -1.58632148, -1.4053405 ],
       [-0.88640526, -0.27630176, -1.58632148,  1.89449216],
       [-0.88640526,  1.37433211, -1.54815205, -1.36651894],
       [-0.88640526, -1.06573534, -1.54815205,  1.04041783],
       [ 1.12815215, -0.13276838, -1.54815205, -1.44416206],
       [ 1.12815215, -1.20926872, -1.54815205,  1.11806095],
       [-0.88640526, -0.